In [2]:
import numpy as np
import cv2
from PIL import Image
import os


# **Face Recognition**

use openCV2 to recognise the face and eyes using camera

In [3]:
import numpy as np
import cv2

# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades

#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml
face_cascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
#https://github.com/Itseez/opencv/blob/master/data/haarcascades/haarcascade_eye.xml
eye_cascade = cv2.CascadeClassifier('Cascades/haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

while 1:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(20, 20)
    )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img',frame)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

# **Dataset creation**
create a dataset of faces by opening the camera for 10 seconds taking 40 pictures (30 for training and 10 for testing) using live camera

In [5]:
import cv2
import os
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
face_detector = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')# For each person, enter one numeric face trueLabel
# For each person, enter one numeric face trueLabel
face_id = input('\n enter user trueLabel end press <return> ==>  ')
print("\n [INFO] Initializing face capture. Look the camera and wait ...")# Initialize individual sampling face count
count = 0
ESCAPE_KEY = 27
trainingDataNumber = 30
testingDataNumber = 10
currentLabel = 'train'

while(True):
    ret, img = cam.read()
    #img = cv2.flip(img, -1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        #image saved as User.trueLabel.count (User.1.14 for example)
        cv2.imwrite("dataset/User." + str(face_id) + '.' +  
                    str(count) +'_'+ currentLabel +  ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    
    # Waiting for 250 miliseconds between every 2 captures
    k = cv2.waitKey(250) & 0xff
    # Press 'ESC' for exiting video 
    if k == ESCAPE_KEY:
        break
    # Take 40 face samples and stop video
    elif count >= trainingDataNumber + testingDataNumber:
         break
    # Take 30 face samples for training and change the label in the image path to 'test'
    elif count >= trainingDataNumber:
        currentLabel = 'test'
    
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


# **Training**
Training dataset using LBPH Face Recognizer

In [10]:
import cv2
import numpy as np
from PIL import Image
import os
# Path for face image database
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("Cascades/haarcascade_frontalface_default.xml")
# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    # faceSamples is the list of every face in every image  
    faceSamples=[]
    # True labels of the faces
    ids = []    
    # Loop through all the image paths which contains 'train' label in it
    # and loading each image
    for imagePath in imagePaths:
        if(imagePath.__contains__('test')):
            break
        PIL_img = Image.open(imagePath).convert('L') # grayscale
        img_numpy = np.array(PIL_img,'uint8')
        
        trueLabel = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(trueLabel)
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)

# train the model using the faces and ids
recognizer.train(faces, np.array(ids))# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') # Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 2 faces trained. Exiting Program


# **Live Recognizer**
Demo for the live recognizer which detects the faces with the live camera and add the label of the face on it.

In [7]:
import cv2
import numpy as np
import os 
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX
ESCAPE_KEY = 27
# names related to ids: example ==> Marcelo: trueLabel=1,  etc
names = ['None', 'Barbary', 'Tython','Messi', 'Adam', 'Mahmoud', 'sary', 'sary', 'sary','saryyyyy', 'sary', 'sary', 'sary'] # Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height
# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
while True:
    ret, img =cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )
    # Loop through each face in the image
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        trueLabel, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # If confidence is less them 75 then matchs the trueLabel 
        if (confidence < 75):
            trueLabel = names[trueLabel]
            confidence = "  {0}%".format(round(100 - confidence))
        # else this face is unknown
        else:
            trueLabel = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    img, 
                    str(trueLabel), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff
    # Press 'ESC' for exiting video
    if k == ESCAPE_KEY:
        break
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff


# **testing**
Testing the model on unseen data.

In [11]:
import cv2
import numpy as np
from PIL import Image
import os
from sklearn.metrics import accuracy_score

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
path = 'dataset'
imagePaths = [os.path.join(path, f) for f in os.listdir(path)]

# Split the dataset into training and testing sets
train_imagePaths = [img for img in imagePaths if 'train' in img]
test_imagePaths = [img for img in imagePaths if 'test' in img]

y_true = []
y_pred = []
# Loop through the test images
for imagePath in test_imagePaths:
    PIL_img = Image.open(imagePath).convert('L')  # grayscale
    img_numpy = np.array(PIL_img, 'uint8')
    trueLabel = int(os.path.split(imagePath)[-1].split(".")[1])
    # Detect the face in the image
    faces = faceCascade.detectMultiScale(
        img_numpy,
        scaleFactor=1.2,
        minNeighbors=5
    )
    # Loop through each face in the image
    for (x, y, w, h) in faces:
        predictedLabel, confidence = recognizer.predict(img_numpy[y:y + h, x:x + w])
        # Append the true label and the predicted label in the lists
        y_true.append(trueLabel)
        y_pred.append(predictedLabel)

        if (confidence < 75):
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            confidence = "  {0}%".format(round(100 - confidence))

        print(f"True Label: {trueLabel}, Predicted Label: {predictedLabel}, Confidence: {confidence}")

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")


True Label: 10, Predicted Label: 10, Confidence:   67%
True Label: 10, Predicted Label: 10, Confidence:   60%
True Label: 10, Predicted Label: 10, Confidence:   59%
True Label: 10, Predicted Label: 10, Confidence:   61%
True Label: 10, Predicted Label: 10, Confidence:   54%
Accuracy: 1.0
